In [1]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:95% !important; }</style>"))
%config IPCompleter.use_jedi=False

In [151]:
import json
import requests
from time import sleep

import pandas as pd
from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login
from tqdm.notebook import tqdm

In [3]:
notebook_login()

Login successful
Your token has been saved to /home/jovyan/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [15]:
API_TOKEN = 'hf_XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' #Replace with token
API_URL = "https://api-inference.huggingface.co/models/bigscience/T0pp"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

In [9]:
dataset = load_dataset('wili_2018', split="train+test", ignore_verifications=True, cache_dir='../data/cache/')
hipe_languages_dataset = dataset.filter(filter_en_fr_de)

Reusing dataset wili_2018 (../data/cache/wili_2018/WiLI-2018 dataset/1.1.0/78d7fe4a9d0a01168e45657f302c776ee0afc0978d44e2c3759f4c4975b845f5)


In [184]:
def query(payload):
    data = json.dumps(payload)
    response = requests.request("POST", API_URL, headers=headers, data=data)
    data = json.loads(response.content.decode("utf-8"))
    try:
        return data[0].get("generated_text")
    except:
        return 0
    
def prompt_text_language_prediction(sentence):
    return f"{sentence}\nQ: Name the language of the previous sentence.\nA:"

def prompt_text_author_prediction(sentence):
    return f"Paragraph:\"{sentence[:500]}\"\nQ:What famous author wrote the previous paragraph?\nA:"

def filter_en_fr_de(example,):
    l = example['label']
    return (l==dataset.features['label'].str2int('eng') or l==dataset.features['label'].str2int('fra') or l==dataset.features['label'].str2int('deu'))

def predict_language(sentence):
    return query(prompt_text_language_prediction(sentence.split('.')[0]))

def predict_author(sentence):
    return query(prompt_text_author_prediction(sentence)).

In [36]:
hipe_languages_dataset[40]

{'sentence': 'West Coast Reggae Group Stick Figure Debuts First-Ever Music Video for \'Fire on the Horizon\' Track: Exclusive Premiere"". Billboard. November 10, 2015.',
 'label': 41}

In [48]:
preds = []
for s in tqdm(hipe_languages_dataset['sentence']):
    sleep(3)
    preds.append(predict_language(s))

  0%|          | 0/3000 [00:00<?, ?it/s]

In [101]:
df = hipe_languages_dataset.to_pandas()
df['preds'] = preds
df['preds'] = df['preds'].str.lower()

In [103]:
def format_predictions(guess):
    if guess in english:
        return 'eng'
    elif guess in french:
        return 'fra'
    elif guess in german:
        return 'deu'
    else:
        return guess
    
english = [
'american english',
'anglais',
'australian english',
'british english',
'en',
'englisch',
'english',
'englsh',
'nevada english',
]

french = [
'francais',
'französisch',
'french',
'swiss french',
]


german = [
 'german',
 'swiss german',
]

In [104]:
df['preds']  = df['preds'].apply(format_predictions)
df['label'] = df['label'].apply(hipe_languages_dataset.features['label'].int2str)

In [188]:
df.to_json('./final_preds.jsonl', orient='records', lines=True)

In [113]:
sum(df['label'] == df['preds'])/len(df) #TOTAL ACCURACY

0.6416666666666667

In [109]:
fr = df[df['label']=='fra']
en = df[df['label']=='eng']
de = df[df['label']=='deu']

In [115]:
sum(fr['label'] == fr['preds'])/len(fr) #French ACCURACY

0.83

In [114]:
sum(en['label'] == en['preds'])/len(en) #English ACCURACY

0.354

In [117]:
sum(de['label'] == de['preds'])/len(de) #German ACCURACY

0.741

In [86]:
df['preds'].value_counts()

fra                        887
deu                        762
eng                        400
latin                      278
russian                    166
                          ... 
cairo                        1
nepali                       1
ethiopian                    1
gaelic                       1
catalinbread antichthon      1
Name: preds, Length: 118, dtype: int64

In [148]:
authorship_df = pd.read_csv('./dataset/Gungor_2018_VictorianAuthorAttribution_data-train.csv', encoding='iso-8859-1')
authors = pd.read_csv('./dataset/authors.txt', header=None)
authors.index = pd.RangeIndex(start=1, stop=51, step=1)
author_dict = authors[0].to_dict()
authorship_df['author_name'] = authorship_df['author'].apply(lambda x: author_dict[x])
authorship_dataset = Dataset.from_pandas(authorship_df)

In [149]:
authorship_df

,text,author,author_name
0,ou have time to listen i will give you the ent...,1,Arthur Conan Doyle
1,wish for solitude he was twenty years of age a...,1,Arthur Conan Doyle
2,and the skirt blew in perfect freedom about th...,1,Arthur Conan Doyle
3,of san and the rows of shops opposite impresse...,1,Arthur Conan Doyle
4,an hour s walk was as tiresome as three in a s...,1,Arthur Conan Doyle
...,...,...,...
53673,after surrounding and searching the whole plac...,50,William Black
53674,giant who could make a young earthquake or a w...,50,William Black
53675,waters of the lake at the bottom of the hill c...,50,William Black
53676,fingers and thumb in it exactly as it came out...,50,William Black


In [194]:
authors = []
for s in tqdm(authorship_dataset.shuffle()['text'][:1000]):
    sleep(1)
    authors.append(predict_language(s))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [195]:
authors

["george's plan",
 "emperor's army",
 'i sat down to write',
 'mrs ood went to the door and opened it for mrs wilson',
 'horses that are to be driven in a coach ought to be driven at the cheek of an easy bit with their pole piece rather slack',
 'mr love gone',
 'i am not thinking of you she said i am thinking of that man my father well and that way too said l i can be of use to you that way too i will have to be it is very needful my dear that we should consult about your father for the way this talk has',
 'william',
 's',
 'a system whose is the exploitation of the people of this country',
 'a universal whisper ran through the ai the old ladies all felt shocked in the extreme ladies blushed and felt excessively for the and even the nor himself was observed to b troubled in mind to complete the good folks',
 'No',
 'the solitary student',
 'i could quote many more passages from i that respecting the ruins of rome but i think you will agree that this is a sufficient specimen of the po